In [1]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream

from alpaca.data.requests import (
    StockBarsRequest,
    StockTradesRequest,
    StockQuotesRequest
)
from alpaca.trading.requests import (
    GetAssetsRequest, 
    MarketOrderRequest, 
    LimitOrderRequest, 
    StopOrderRequest, 
    StopLimitOrderRequest, 
    TakeProfitRequest, 
    StopLossRequest, 
    TrailingStopOrderRequest, 
    GetOrdersRequest, 
    ClosePositionRequest
)
from alpaca.trading.enums import ( 
    AssetStatus, 
    AssetExchange, 
    OrderSide, 
    OrderType, 
    TimeInForce, 
    OrderClass, 
    QueryOrderStatus
)
from alpaca.common.exceptions import APIError

In [2]:
api_key = ""
secret_key = ""

paper = True 

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

# Fetching Market Data

In [3]:
# setup stock historical data client
stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [27]:
symbol = "TSLA"
now = datetime.now(ZoneInfo("America/New_York"))
start_date = datetime(2022, 9, 1)  # Starting from September 1, 2022
end_date = datetime(2024, 9, 1)    # Ending on September 1, 2024

In [29]:
req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 2, unit = TimeFrameUnit.Hour), # specify timeframe
    start = start_date,                          # specify start datetime, default=the beginning of the current day.
    end_date=end_date,                                        # specify end datetime, default=now                                               # specify limit
)
stock_historical_data_client.get_stock_bars(req).df

open     high       low   close  \
symbol timestamp                                                      
TSLA   2022-09-01 00:00:00+00:00  271.57  273.800  268.9200  273.00   
       2022-09-01 12:00:00+00:00  271.60  280.340  266.1500  279.70   
       2022-09-02 00:00:00+00:00  279.10  280.000  277.2900  278.15   
       2022-09-02 12:00:00+00:00  279.70  282.570  269.0800  269.80   
       2022-09-06 00:00:00+00:00  276.14  276.140  269.3300  272.52   
...                                  ...      ...       ...     ...   
       2024-10-28 12:00:00+00:00  269.68  273.536  260.0200  260.10   
       2024-10-29 00:00:00+00:00  262.00  263.490  260.7700  262.88   
       2024-10-29 12:00:00+00:00  262.68  264.980  255.5100  259.80   
       2024-10-30 00:00:00+00:00  260.00  261.990  258.8100  258.81   
       2024-10-30 12:00:00+00:00  260.02  263.350  255.8201  256.05   

                                       volume  trade_count        vwap  
symbol timestamp                                                        
TSLA   2022-09-01 00:00:00+00:00     474293.0      14682.0  271.066567  
       2022-09-01 12:00:00+00:00   55295026.0     818834.0  271.615726  
       2022-09-02 00:00:00+00:00     217908.0       6591.0  278.527202  
       2022-09-02 12:00:00+00:00   52276885.0     746352.0  274.392107  
       2022-09-06 00:00:00+00:00     490282.0      12595.0  272.805371  
...                                       ...          ...         ...  
       2024-10-28 12:00:00+00:00  106059453.0    1368849.0  268.262230  
       2024-10-29 00:00:00+00:00     750985.0      21509.0  262.196046  
       2024-10-29 12:00:00+00:00   79762934.0    1077481.0  259.519175  
       2024-10-30 00:00:00+00:00     556408.0      15559.0  260.453900  
       2024-10-30 12:00:00+00:00   53129615.0     804275.0  260.379668  

[1107 rows x 7 columns]